In [10]:
import pandas as pd
from sklearn.datasets import fetch_openml
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import keras
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import os
import urllib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype(np.int64)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X.shape, X_test.shape

((70000, 784), (14000, 784))

In [5]:
# Função para calcular distâncias pareadas
def pairwise_distances(x, y):
    x_norm = tf.reduce_sum(tf.square(x), axis=1, keepdims=True)
    y_norm = tf.reduce_sum(tf.square(y), axis=1, keepdims=True)
    dists = x_norm - 2 * tf.matmul(x, y, transpose_b=True) + tf.transpose(y_norm)
    return tf.sqrt(tf.maximum(dists, 0.0))  # Garantir que as distâncias não sejam negativas

In [12]:
# Convertendo os dados para tensores TensorFlow
tf_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
tf_train = tf.convert_to_tensor(X, dtype=tf.float32)

# Criando o dataset com o tf.data
ds = tf.data.Dataset.from_tensor_slices(tf_test).batch(1000)  # Definindo batch_size

_min_dists = []
_arg_min_dists = []

# Criando o iterador do dataset
iterator = iter(ds)

while True:
    try:
        batch = next(iterator)  # Obtém o próximo batch
        dists = pairwise_distances(tf_train, batch)
        min_dist = tf.reduce_min(dists, axis=0)
        arg_min_dist = tf.argmin(dists, axis=0)
        _min_dists.append(min_dist)
        _arg_min_dists.append(arg_min_dist)
    except StopIteration:
        break  # Sai do loop quando os dados acabam

2025-01-27 11:26:32.608684: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [13]:
# Concatenando os resultados
_min_dists = tf.concat(_min_dists, axis=0)
_arg_min_dists = tf.concat(_arg_min_dists, axis=0)

In [14]:
y_pred = y_train[_arg_min_dists]

# Calculer l'accuracy
accuracy = accuracy_score(y_test, y_pred)

IndexError: index 56364 is out of bounds for axis 0 with size 56000